In [1]:
from src.data_loader.data_loader import BooksDataset
from src.models.mm_model import MmModel
import pandas as pd
from src.train import Trainer
import torch

In [ ]:
# make auto reload
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = BooksDataset(data_dir="../data/books")

In [4]:
model = MmModel(n_users=dataset.n_users,n_items=dataset.n_items,adjacency_matrix=dataset.get_dataset("adjacency_matrix"),embed_size=64,n_layers=3)

In [5]:
trainer = Trainer(model=model,dataset=dataset)

In [7]:
trainer.train(epochs=2,batch_size=1024)

100%|██████████| 15/15 [00:07<00:00,  1.92it/s]


{'precision': array([0.00903819, 0.00764847, 0.00563087]), 'recall': array([0.01540955, 0.02586903, 0.04656824]), 'ndcg': array([0.03485256, 0.04540221, 0.06153235]), 'hit_ratio': array([0.07458508, 0.11177764, 0.16496701]), 'auc': 0.0}
Epoch 0 Loss 0.6931116183598837 Time 54.91503047943115


100%|██████████| 15/15 [00:08<00:00,  1.73it/s]


{'precision': array([0.00959808, 0.00804839, 0.00585883]), 'recall': array([0.01630843, 0.02759412, 0.04839477]), 'ndcg': array([0.03518603, 0.04613352, 0.06311885]), 'hit_ratio': array([0.0779844 , 0.11737652, 0.17456509]), 'auc': 0.0}
Epoch 1 Loss 0.6930795907974243 Time 56.11346197128296
